In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Hyperparameters


In [ ]:

image_size = (224, 224)
batch_size = 32
num_classes = 5  # categories in last output layer
learning_rate = 0.001 #when u adjust the weights the difference in adjustments will be this much
epochs = 10

In [ ]:
import os
import zipfile
import shutil
import pandas as pd
import numpy as np

# Unzip the training and testing image files
# with zipfile.ZipFile('train_images.zip', 'r') as zip_ref:
#     zip_ref.extractall('train_images')

# with zipfile.ZipFile('test_images.zip', 'r') as zip_ref:
#     zip_ref.extractall('test_images')

# # Load CSV and create class folders
# csv_file = 'train.csv'  # Make sure this file exists in your working directory
# df = pd.read_csv(csv_file)

# base_dir = 'dataset'
# os.makedirs(base_dir, exist_ok=True)

# for level in df['diagnosis'].unique():
#     os.makedirs(os.path.join(base_dir, str(level)), exist_ok=True)

# for _, row in df.iterrows():
#     src = os.path.join('train_images', row['id_code'] + '.png')
#     dst = os.path.join(base_dir, str(row['diagnosis']), row['id_code'] + '.png')
#     if os.path.exists(src):
#         shutil.copy(src,dst)

train data set


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 80% Train, 20% Validation
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'colored_images',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

testing

In [ ]:
validation_generator = train_datagen.flow_from_directory(
    'colored_images',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    'colored_images',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

model training


In [ ]:
base_model = ResNet50(
    weights='imagenet', 
    include_top=False, 
    input_shape=(224, 224, 3)
)

# Freeze base model layers (Only train custom layers)
base_model.trainable = False

# Custom Fully Connected Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks (Model Checkpoint & Early Stopping)
checkpoint = ModelCheckpoint(
    'best_resnet50_model.keras', 
    monitor='val_accuracy', 
    save_best_only=True
)

early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5,  # Reduce to avoid long training in Kaggle
    restore_best_weights=True
)

train model

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

#  Callbacks (Model Checkpoint & Early Stopping)
checkpoint = ModelCheckpoint(
    'best_resnet50_model.keras', 
    monitor='val_accuracy', 
    save_best_only=True
)

early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5,  # Reduce to avoid long training in Kaggle
    restore_best_weights=True
)

In [ ]:
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[checkpoint, early_stopping]
)

In [ ]:
test_loss, test_acc = model.evaluate(validation_generator)
print(f" Test Accuracy: {test_acc:.4f}")


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Class Labels (Make sure the order matches your folder names)
labels = {
    0: "No DR",
    1: "Mild",
    2: "Moderate",
    3: "Severe",
    4: "Proliferative DR"
}

# Load and Predict Function
def load_and_predict(image_path):

    # Load the pre-trained model
    # loaded_model = tf.keras.models.load_model("retino_cnn_model.h5")
    # print("Model loaded successfully.")

    # Load and preprocess the image
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = image / 255.0
    image = np.expand_dims(image, axis=0)

    # Make predictions
    prediction =model.predict(image)
    predicted_class = np.argmax(prediction, axis=1)[0]

    # Map to class label and display
    category = labels[predicted_class]
    print(f"Predicted Class: {predicted_class} ({category})")
    return category

# Example usage:
load_and_predict("00cc2b75cddd.png")


In [ ]:
model.save('final_model.h5')

print("Model saved as final_model.h5")